<a href="https://colab.research.google.com/github/Alvaro-23-N/Python_Ingenira/blob/main/Evaluacion_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Crear un sistema basico de robot**

Parte 1: Clase Base Robot

In [ ]:
class Robot:
  #ARTIBUTOS DE INSTANCIA
    def __init__(self, name: str, model: str):
        self.name = name
        self.model = model
        self.battery = 100.0  # NIVEL DE BARERIA (%)
        self.distance_traveled = 0  # DISTANCIA TOTAL (Km)
        self.speed = 0  # VELOCIDAD ACTUAL (Km/h)
        self.direction = 0  # DIRECCION (0-359°)

        print(f"Robot {name} modelo {model} creado")

    def __str__(self):

        return f"{self.name} ({self.model}) - Batería: {self.battery:.1f}% - Distancia: {self.distance_traveled:.2f}km"

    def move(self, speed: float, direction: int, time_minutes: float):

        # Actualizar velocidad y dirección
        self.speed = speed
        self.direction = direction % 360

        # Calcular distancia: distancia = speed * (time_minutes / 60)
        time_hours = time_minutes / 60.0

        #Sumar a distance_traveled
        distance = speed * time_hours
        self.distance_traveled += distance

        #Reducir batería: batería -= distancia * 10
        battery_consumption = distance * 10
        self.battery -= battery_consumption

        # Verificar batería baja
        if 0 <= self.battery < 20:
            print("¡Batería baja!")

    def charge(self, time_minutes: float):

        # Aumentar batería: batería += time_minutes * 2
        battery_increase = time_minutes * 2

        # Aumentar batería máximo 100%
        new_battery = self.battery + battery_increase
        self.battery = min(new_battery, 100.0)

        print(f"{self.name} cargando... Batería: {self.battery:.1f}%")

    def get_status(self):

        print(f"\n{'='*50}\nESTADO DE {self.name.upper()}\n{'='*50}\nNombre: {self.name}\nModelo: {self.model}\nBatería: {self.battery:.1f}%\nDistancia recorrida: {self.distance_traveled:.2f} km\nVelocidad actual: {self.speed} km/h\nDirección actual: {self.direction}°")



Parte 2:Herencia - Tipos de Robot

In [ ]:
class WorkerRobot(Robot):

    def __init__(self, name, model):
        super().__init__(name, model)
        self.cargo = 0  # Carga actual en kg

    def move(self, distance):

        if self.battery > 0:
            # Calcular consumo base
            battery_consumption = distance * 10

            # Si tiene carga, aumentar consumo en 20%
            if self.cargo > 0:
                battery_consumption *= 1.2
                print(f"{self.name} consume 20% más batería por llevar {self.cargo}kg de carga")

            if self.battery >= battery_consumption:
                self.battery -= battery_consumption
                self.distance_traveled += distance
                print(f"{self.name} se movió {distance}km. Batería: {self.battery:.1f}%")
            else:
                print(f"{self.name} no tiene suficiente batería para moverse {distance}km con {self.cargo}kg de carga")
        else:
            print(f"{self.name} necesita carga para moverse")

    def load_cargo(self, weight):

        self.cargo += weight
        print(f"Cargando {weight}kg. Total: {self.cargo}kg")

    def unload_cargo(self):

        self.cargo = 0
        print("Carga descargada")

class GuardRobot(Robot):

    def __init__(self, name, model):
        super().__init__(name, model)
        self.alert_mode = False  # Modo alerta inicialmente desactivado

    def charge(self, time_minutes):
      # Define la eficiencia según el modo alerta
      efficiency = 1 if self.alert_mode else 2

      if self.alert_mode:
        print(f"{self.name} está en modo alerta - Advertencia: carga 50% más lento")

      # Calcula la carga incrementada de batería
      battery_increase = time_minutes * efficiency
      self.battery = min(self.battery + battery_increase, 100.0)

      print(f"{self.name} cargado por {time_minutes} minutos. Batería: {self.battery:.1f}%")

    def toggle_alert(self):
      #Modo de alerta ON/OFF
        self.alert_mode = not self.alert_mode
        status = "ON" if self.alert_mode else "OFF"
        print(f"Modo alerta: {status}")

    def show_status(self):

        alert_status = "ACTIVADO" if self.alert_mode else "DESACTIVADO"
        print(f"""{'='*50}ESTADO DE {self.name.upper()} (GUARDIA){'='*50}Nombre: {self.name}Modelo: {self.model}Batería: {self.battery:.1f}%Distancia recorrida: {self.distance_traveled:.2f} kmVelocidad actual: {self.speed} km/hDirección actual: {self.direction}°Modo alerta: {alert_status}""")



Parte 3: Composición - Clase RobotTeam

In [ ]:
class RobotTeam:

    def __init__(self, team_name):
        self.team_name = team_name # Nombre del equipo
        self.robots = []  # Lista de robots en el equipo
        self.missions_completed = 0  # Misiones completadas (inicialmente 0)
        print(f"Equipo {team_name} formado")

    def add_robot(self, robot):
      #Añadir robots al equipo
        if isinstance(robot, Robot):
            self.robots.append(robot)
            print(f"{robot.name} se unió al equipo {self.team_name}")
        else:
            print("Error: Solo se pueden agregar objetos Robot al equipo")

    def __len__(self):
      #Retornar número de robots en el equipo
        return len(self.robots)

    def charge_all(self, time_minutes):
      #Cargar todos los robots del equipo simultáneamente
        if not self.robots:
            print(f"No hay robots en el equipo {self.team_name} para cargar")
            return

        print(f"\n Cargando todos los robots del equipo {self.team_name}...")
        print("-" * 40)

        for robot in self.robots:
            robot.charge(time_minutes)

    def team_status(self):
       #Mostrar información de todos los robots
        print(f"""{'='*60}ESTADO DEL EQUIPO: {self.team_name.upper()}{'='*60}Total de robots: {len(self.robots)}Misiones completadas: {self.missions_completed}{'='*60}""")

        if not self.robots:
            print(" No hay robots en este equipo")
            return
        #Mostrar total de robots y misiones completadas
        for i, robot in enumerate(self.robots, 1):
            print(f"\n ROBOT #{i}:")
            robot.show_status()

    def complete_mission(self):
        #Incrementar missions_completed
        self.missions_completed += 1
        print(f"¡Misión completada! Total: {self.missions_completed}")


Parte 4: Código de Prueba

In [ ]:

    robot1 = Robot("Arturito", "R2-D2")
    robot2 = Robot("C3PO", "Protocol Droid")

    # 1 WorkerRobot
    worker = WorkerRobot("WALL-E", "Waste Compactor")

    # 1 GuardRobot
    guard = GuardRobot("Terminator", "T-800")
    print(f"\n Total de robots creados: {Robot.total_robots}")

    ## PROBAR MÉTODOS BÁSICOS
    robots = [robot1, robot2, worker, guard]

    # Mover cada robot (diferentes velocidades/direcciones/tiempos)
    movimientos = [
    {"speed": 25, "direction": 90, "time_minutes": 30},
    {"speed": 15, "direction": 180, "time_minutes": 20},
    {"speed": 20, "direction": 45, "time_minutes": 25},
    {"speed": 30, "direction": 270, "time_minutes": 15},
]
    for robot, params in zip(robots, movimientos):
    robot.move(**params)

    # Cargar cada robot
    tiempos_carga = [10, 15, 20, 12]
    for robot, tiempo in zip(robots, tiempos_carga):
    robot.charge(tiempo)

    # Mostrar status de cada robot
    for robot in robots:
    robot.get_status()

    # Probar funciones especiales
    # WorkerRobot
    worker.load_cargo(50)
    worker.load_cargo(25)
    worker.move(speed=18, direction=135, time_minutes=20)  # Consume más batería
    worker.unload_cargo()

    # GuardRobot
      guard.toggle_alert()  # Activar alerta
      guard.charge(10)      # Carga más lenta en modo alerta
      guard.toggle_alert()  # Desactivar alerta
      guard.charge(10)      # Carga normal

    # Crear un RobotTeam
    team = RobotTeam("Team Omega")

    # Agregar todos los robots al equipo
    print("\n👥 Formando equipo:")
    team.add_robot(robot1)
    team.add_robot(robot2)
    team.add_robot(worker)
    team.add_robot(guard)

    numero_robots = len(robot_team.robots)
    print(f"El equipo tiene {numero_robots} robots")

    # Cargar todo el equipo
    team.charge_all(15)

    # Mostrar status del equipo
    team.team_status()

    # Completar una misión
    team.complete_mission()
    team.complete_mission()
    team.complete_mission()

    team.team_status()

CODIGO COMPLETO

In [6]:
from inspect import FullArgSpec
class Robot:
    total_robots = 0
    max_battery = 100.0
    min_battery = 0.0

    def __init__(self, name: str, model: str):
        self.name = name
        self.model = model
        self.battery = self.max_battery
        self.distance_traveled = 0.0
        self.speed = 0
        self.direction = 0
        Robot.total_robots += 1

        print(f"Robot {name} modelo {model} creado")

    def __str__(self):
        return f"{self.name} ({self.model}) - Batería: {self.battery:.1f}% - Distancia: {self.distance_traveled:.2f}km"

    def move(self, speed: float, direction: int, time_minutes: float):
      #MUEVE EL ROBOT CONSIDERANDO SU CONSUMO ENERGETICO
      #SEGUN LA VELOCIDAD Y EL TIEMPO EN MOVIMIENTO CONSUMIRA MAS O MENOS BATERIA

    # Validar y actualizar estado
      if not (speed >= 0 and time_minutes > 0 and self.battery > 0):
        print(f"{self.name} no puede moverse")
        return False

      self.speed = speed
      self.direction = direction % 360

    # Calcular y validar consumo
      distance = speed * (time_minutes / 60.0)
      battery_needed = distance * 10

      if self.battery < battery_needed:
        print(f"{self.name} batería insuficiente")
        return False

    # Ejecutar movimiento
      self.battery -= battery_needed
      self.distance_traveled += distance

    # Resultado y advertencias
    print(f"{self.name} se movió {distance:.2f}km a {speed}km/h hacia {direction}°. Batería: {self.battery:.1f}%")

    if self.battery < 20:
        print("¡Batería baja!")

    return True

    def charge(self, time_minutes: float):
      #LA CARGA DE LA BATERIA DEL ROBOT ES PERSONALIZABLE
        battery_increase = time_minutes * 2
        new_battery = self.battery + battery_increase
        self.battery = min(new_battery, 100.0)

        print(f"{self.name} cargando... Batería: {self.battery:.1f}%")

    def get_status(self):
        print(f"""
{'='*50}
ESTADO DE {self.name.upper()}
{'='*50}
Nombre: {self.name}
Modelo: {self.model}
Batería: {self.battery:.1f}%
Distancia recorrida: {self.distance_traveled:.2f} km
Velocidad actual: {self.speed} km/h
Dirección actual: {self.direction}°""")


class WorkerRobot(Robot):
  max_capacity=200 # Kg
  maintenace_interval=100 # Km

  def __init__(self, name, model):
    super().__init__(name, model)

    if capacity > self.max_capacity:
      raise ValueError(f"La carga máxima de {self.name} es {self.max_capacity}kg")

    self.cargo_weight = 0
    self.max_capacity=capacity
    self.maintenace_interval=False
    self.km_since_maintenance=0
    self.cargo_type=None

  def load_cargo(self, weight:float, cargo_type:str):
    if weight<=0:
      return False
    print(f" {self.name} cargó {weight}kg de {cargo_type}. Total: {self.cargo_weight}kg")

    if self.cargo_weight + weight > self.max_capacity
      return False

    self.cargo_weight += weight
    self.cargo_type=cargo_type

  def unload_cargo(self):
    if self.cargo_weight == 0:
      return 0

    unloaded_weight=self.cargo_weight
    cargo_type=self.cargo_type
    print(f" {self.name} descargó {unloaded_weight}kg de {cargo_type}")

  def move(self, speed:float, direction:int, time_minutes:float):
    if self.maintenance_required:
      return 0
    max_speed_with_load=50*(1-self.cargo_weight/self.max_capacity)
    if speed > max_speed_with_load:
                  speed = max_speed_with_load


    distance = super().move(speed, direction, time_minutes)

    if distance > 0:

            self.km_since_maintenance += distance
            if self.km_since_maintenance >= self.MAINTENANCE_INTERVAL:
                self.maintenance_required = True
                print(f"🔧 {self.name} necesita mantenimiento después de {self.km_since_maintenance}km")


class GuardRobot(Robot):
    def __init__(self, name, model):
        super().__init__(name, model)
        self.alert_mode = False

    def charge(self, time_minutes):
        efficiency = 1 if self.alert_mode else 2

        if self.alert_mode:
            print(f"{self.name} está en modo alerta - Advertencia: carga 50% más lento")

        battery_increase = time_minutes * efficiency
        self.battery = min(self.battery + battery_increase, 100.0)

        print(f"{self.name} cargado por {time_minutes} minutos. Batería: {self.battery:.1f}%")

    def toggle_alert(self):
        self.alert_mode = not self.alert_mode
        status = "ON" if self.alert_mode else "OFF"
        print(f"Modo alerta: {status}")

    def get_status(self):
        alert_status = "ACTIVADO" if self.alert_mode else "DESACTIVADO"
        print(f"""
{'='*50}
ESTADO DE {self.name.upper()} (GUARDIA)
{'='*50}
Nombre: {self.name}
Modelo: {self.model}
Batería: {self.battery:.1f}%
Distancia recorrida: {self.distance_traveled:.2f} km
Velocidad actual: {self.speed} km/h
Dirección actual: {self.direction}°
Modo alerta: {alert_status}""")


class RobotTeam:
    def __init__(self, team_name):
        self.team_name = team_name
        self.robots = []
        self.missions_completed = 0
        print(f"Equipo {team_name} formado")

    def add_robot(self, robot):
        if isinstance(robot, Robot):
            self.robots.append(robot)
            print(f"{robot.name} se unió al equipo {self.team_name}")
        else:
            print("Error: Solo se pueden agregar objetos Robot al equipo")

    def __len__(self):
        return len(self.robots)

    def charge_all(self, time_minutes):
        if not self.robots:
            print(f"No hay robots en el equipo {self.team_name} para cargar")
            return

        print(f"\nCargando todos los robots del equipo {self.team_name}...")
        print("-" * 40)

        for robot in self.robots:
            robot.charge(time_minutes)

    def team_status(self):
        print(f"""
{'='*60}
ESTADO DEL EQUIPO: {self.team_name.upper()}
{'='*60}
Total de robots: {len(self.robots)}
Misiones completadas: {self.missions_completed}
{'='*60}""")

        if not self.robots:
            print("No hay robots en este equipo")
            return

        for i, robot in enumerate(self.robots, 1):
            print(f"\nROBOT #{i}:")
            robot.get_status()

    def complete_mission(self):
        self.missions_completed += 1
        print(f"¡Misión completada! Total: {self.missions_completed}")


if __name__ == "__main__":
    robot1 = Robot("Arturito", "R2-D2")
    robot2 = Robot("Svarog", "Geomarrow. ")
    worker = WorkerRobot("WALL-E", "Waste Compactor")
    guard = GuardRobot("Terminator", "T-800")

    print(f"\nTotal de robots creados: {Robot.total_robots}")

    robots = [robot1, robot2, worker, guard]

    movements = [
        {"speed": 25, "direction": 90, "time_minutes": 30},
        {"speed": 15, "direction": 180, "time_minutes": 20},
        {"speed": 20, "direction": 45, "time_minutes": 25},
        {"speed": 30, "direction": 270, "time_minutes": 15},
    ]

    print("\nMoviendo robots:")
    for robot, params in zip(robots, movements):
        robot.move(**params)

    loading_times = [10, 15, 20, 12]
    print("\nCargando robots:")
    for robot, tiempo in zip(robots, loading_times):
        robot.charge(tiempo)

    print("\nEstado de todos los robots:")
    for robot in robots:
        robot.get_status()

    print("\nProbando funciones especiales:")
    worker.load_cargo(50)
    worker.load_cargo(25)
    worker.move(speed=18, direction=135, time_minutes=20)
    worker.unload_cargo()

    guard.toggle_alert()
    guard.charge(10)
    guard.toggle_alert()
    guard.charge(10)

    team = RobotTeam("Team Omega")

    print("\nFormando equipo:")
    team.add_robot(robot1)
    team.add_robot(robot2)
    team.add_robot(worker)
    team.add_robot(guard)

    numero_robots = len(team)
    print(f"El equipo tiene {numero_robots} robots")

    team.charge_all(15)

    print("\nEstado del equipo:")
    team.team_status()

    print("\nCompletando misiones:")
    team.complete_mission()
    team.complete_mission()
    team.complete_mission()

    print("\nEstado final del equipo:")
    team.team_status()

Robot Arturito modelo R2-D2 creado
Robot Svarog modelo Geomarrow.  creado
Robot WALL-E modelo Waste Compactor creado
Robot Terminator modelo T-800 creado

Total de robots creados: 4

Moviendo robots:
Arturito se movió 12.50km a 25km/h hacia 90°. Batería: -25.0%
Svarog se movió 5.00km a 15km/h hacia 180°. Batería: 50.0%
WALL-E se movió 8.33km con 0kg de carga. Batería: 16.7%
¡Batería baja!
Terminator se movió 7.50km a 30km/h hacia 270°. Batería: 25.0%

Cargando robots:
Arturito cargando... Batería: -5.0%
Svarog cargando... Batería: 80.0%
WALL-E cargando... Batería: 56.7%
Terminator cargado por 12 minutos. Batería: 49.0%

Estado de todos los robots:

ESTADO DE ARTURITO
Nombre: Arturito
Modelo: R2-D2
Batería: -5.0%
Distancia recorrida: 12.50 km
Velocidad actual: 25 km/h
Dirección actual: 90°

ESTADO DE SVAROG
Nombre: Svarog
Modelo: Geomarrow. 
Batería: 80.0%
Distancia recorrida: 5.00 km
Velocidad actual: 15 km/h
Dirección actual: 180°

ESTADO DE WALL-E (TRABAJADOR)
Nombre: WALL-E
Modelo: 